In [ ]:
import pandas as pd
import numpy as np
#AUTOR: ALDO MUÑOZ ZECUA

In [ ]:
data = pd.read_csv('da_carpetas-de-investigacion-pgj-cdmx.csv')

In [ ]:
data["delito"].value_counts()

VIOLENCIA FAMILIAR                                 69517
ROBO DE OBJETOS                                    52214
ROBO A NEGOCIO SIN VIOLENCIA                       51426
FRAUDE                                             45349
DENUNCIA DE HECHOS                                 44433
                                                   ...  
PROCREACION ASISTIDA                                   1
EXHORTOS                                               1
HOMICIDIO INTENCIONAL Y ROBO DE VEHICULO               1
ROBO A PASAJERO EN TREN SUBURBANO CON VIOLENCIA        1
CONTRA LA LEY GENERAL DE EXPLOSIVOS                    1
Name: delito, Length: 274, dtype: int64

In [ ]:
data["delito"].unique()

array(['LESIONES CULPOSAS',
       'ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VIOLENCIA',
       'ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA', 'FRAUDE',
       'ROBO A REPARTIDOR CON VIOLENCIA',
       'ROBO A TRANSEUNTE DE CELULAR CON VIOLENCIA', 'ROBO DE DINERO',
       'LESIONES INTENCIONALES', 'ABUSO DE AUTORIDAD',
       'DAÑO EN PROPIEDAD AJENA CULPOSA POR TRÁNSITO VEHICULAR A AUTOMOVIL',
       'ROBO DE VEHICULO DE SERVICIO PÚBLICO SIN VIOLENCIA',
       'DELITOS AMBIENTALES', 'ROBO DE OBJETOS',
       'DAÑO EN PROPIEDAD AJENA INTENCIONAL A AUTOMOVIL',
       'VIOLENCIA FAMILIAR', 'NARCOMENUDEO POSESION SIMPLE',
       'ROBO DE ACCESORIOS DE AUTO',
       'PRIV. ILEGAL DE LA LIB. Y ROBO DE VEHICULO',
       'HOMICIDIO POR ARMA DE FUEGO', 'ROBO A NEGOCIO SIN VIOLENCIA',
       'ROBO DE VEHICULO DE SERVICIO PARTICULAR CON VIOLENCIA',
       'LESIONES CULPOSAS POR TRANSITO VEHICULAR EN COLISION',
       'ABANDONO DE PERSONA', 'DAÑO EN PROPIEDAD AJENA CULPOSA',
       'AMENAZA

In [ ]:
ubicaciones=[]
for i in data.values:
  if(i[8]=="ACOSO SEXUAL") and ( not np.isnan(i[17])) and (not np.isnan(i[18])) and (i[17]!=0) and (i[18]!=0):
    ubicaciones.append([i[17], i[18]])


In [ ]:
ubicaciones_df = pd.DataFrame(ubicaciones, columns=['Latitud', 'Longitud'])

In [ ]:
ubicaciones_df.to_csv('ubicaciones_acoso_sexual.csv', index=False)

##AQUI INICIA LA EJECUCION PARA EL LECTOR:

In [ ]:
ubicaciones=pd.read_csv("'ubicaciones_acoso_sexual.csv'")

In [ ]:
ubicaciones=np.array(ubicaciones)

In [ ]:
import numpy as np
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(coord1, coord2, R=6373.0):
    dlon = radians(coord1[1]) - radians(coord2[1])
    dlat = radians(coord1[0]) - radians(coord2[0])
    a = sin(dlat / 2)**2 + cos(coord1[0]) * cos(coord2[0]) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

def create_adjacency_matrix(ubicaciones, threshold=2):
    num_locations = len(ubicaciones)
    G = np.zeros((num_locations, num_locations))
    d = np.zeros(num_locations)

    for i in range(num_locations):
        for j in range(i + 1, num_locations):
            distance = calculate_distance(ubicaciones[i], ubicaciones[j])
            if distance < threshold:
                G[i, j] = 1
                G[j, i] = 1
                d[i] += 1
                d[j] += 1

    return G, d

threshold_distance = 2
adjacency_matrix, degrees = create_adjacency_matrix(ubicaciones, threshold_distance)

print("Adjacency Matrix:")
print(adjacency_matrix)


Adjacency Matrix:
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 1. 0.]]


In [ ]:
Grafo=np.array(adjacency_matrix)

In [ ]:

all_cliques = []

def is_clique(graph, candidate_nodes, size):
    for i in range(1, size):
        for j in range(i + 1, size):
            if graph[candidate_nodes[i]][candidate_nodes[j]] == 0:
                return False
    return True

def cliques_found(graph, start_node, current_size, target_size, candidate_nodes, degrees):
    global all_cliques
    for j in range(start_node, len(degrees) - (target_size - current_size)):
        if degrees[j] >= target_size - 1:
            candidate_nodes[current_size] = j
            if is_clique(graph, candidate_nodes, current_size + 1):
                if current_size < target_size:
                    cliques_found(graph, j, current_size + 1, target_size, candidate_nodes, degrees)
                else:
                    current_clique = []
                    for a in range(1, current_size + 1):
                        current_clique.append(candidate_nodes[a])
                        graph[candidate_nodes[a]] = np.zeros(len(degrees))
                        graph[:, candidate_nodes[a]] = np.zeros(len(degrees))
                        degrees[candidate_nodes[a]] = 0
                    all_cliques.append(current_clique)

degrees = np.sum(Grafo, axis=1)

for clique_size in range(20, 9, -1):
    candidate_nodes = [0] * (clique_size + 1)
    cliques_found(Grafo, 0, 1, clique_size, candidate_nodes, degrees)



In [ ]:
from math import cos, sin, atan2, sqrt, pi

def calculate_centroid(geolocalizaciones):
    lat_sum = 0
    lon_cos_sum = 0
    lon_sin_sum = 0

    for lat, lon in geolocalizaciones:
        lat_rad = float(lat) * pi / 180
        lon_rad = float(lon) * pi / 180
        lon_cos_sum += cos(lon_rad)
        lon_sin_sum += sin(lon_rad)
        lat_sum += lat_rad

    lon_cos_avg = lon_cos_sum / len(geolocalizaciones)
    lon_sin_avg = lon_sin_sum / len(geolocalizaciones)
    lat_avg = lat_sum / len(geolocalizaciones)

    lon_centroid = atan2(lon_sin_avg, lon_cos_avg)
    return lat_avg * 180 / pi, lon_centroid * 180 / pi

In [ ]:
centros_atencion = [calculate_centroid(ubicaciones[clique]) for clique in all_cliques]

In [ ]:
!pip install folium

In [ ]:
lista_tuplas_intercambiada = [(tupla[1], tupla[0]) for tupla in centros_atencion]

# Imprimir el resultado
print(lista_tuplas_intercambiada)

[(19.40331020179092, -99.17368637455647), (19.33858814935921, -99.15768360586404), (19.455800972611236, -99.17667535825413), (19.35608163823218, -99.06259117752992), (19.447391591013435, -99.12536248245421), (19.368483672423444, -99.1021412078648), (19.400143427470372, -99.20841597403596), (19.33127331916038, -99.00309170095092), (19.304168056652387, -99.12317783822142), (19.430794172108126, -99.14164171917551), (19.303348532442556, -99.10991102944807), (19.431548032739904, -99.15856790958016), (19.45202185004394, -99.07749466187911), (19.343458518656114, -99.04896322687996), (19.414811583228598, -99.11384539007516), (19.392606315876215, -99.19153506429681), (19.51206395414593, -99.13859005189208), (19.47348881822079, -99.19921303059891), (19.46797125943609, -99.09710022869665), (19.31709203278639, -99.18144008186096), (19.370788104374668, -99.0864359013123), (19.385960449601228, -99.13916916742268), (19.467786490587617, -99.15458102569946), (19.33935561788946, -99.06680067179606), (19

In [ ]:
import folium

# Coordenadas de la Ciudad de México
lat, lon = 19.4326, -99.1332

# Crear un mapa centrado en la Ciudad de México
mapa_cdmx = folium.Map(location=[lat, lon], zoom_start=12)

# Añadir marcadores al mapa
for coord in lista_tuplas_intercambiada:
    folium.Marker(location=coord).add_to(mapa_cdmx)

# Guardar el mapa como un archivo HTML
mapa_cdmx.save('mapa_cdmx.html')
